In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import ResNet50
from keras.preprocessing.image import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint


import numpy as np
import os

np.random.seed(2017)

from skimage.io import imread_collection, imread
from skimage.transform import resize

filepath='../every.resn50.hdf5'
img_width, img_height = 243, 243

train_data_dir = '../dataset/training_set/smo'
validation_data_dir = '../dataset/test_set/smo'

nb_train_samples = 4521
nb_validation_samples = 770
epochs = 3
batch_size = 16


if __name__ == '__main__':

    
    
    base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg'
                       ,input_tensor=Input((img_width, img_height, 3)))
        
    # - https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py
    
    for layer in base_model.layers:
        layer.trainable = False

    #x = Flatten()(base_model.output)
    x = Dense(2, activation='softmax', name='fc2')(base_model.output) #加入神經元(隱藏層) 2,foftmax
 
    model = Model(base_model.input, x)

    # for tmp in zip([x.name for x in model.layers], range(len(model.layers))):
    #     print(tmp)

    for layer in model.layers[140:]:
        layer.trainable = True

    # load data
    # now_path = os.path.abspath(os.path.curdir)
    # project_path, tmp = os.path.split(now_path)
    # train_path = os.path.join(project_path, 'train')
    #train_path = '/home/zhangzhe/pycharm/dogcat/train'
    train_path ='../dataset/training_set/smo'
    test_path = '../dataset/test_set/smo'

    #n = 25000
    # X = np.zeros((n, 224, 224, 3), dtype=np.uint8)
    # y = np.zeros((n, 2), dtype=np.float32)
    #
    # for i in range(n//2):
    #     X[i] = resize(imread(os.path.join(train_path, 'cat.%d.jpg' % i)), (224, 224, 3))
    #     X[i+n//2] = resize(imread(os.path.join(train_path, 'dog.%d.jpg' % i)), (224, 224, 3))
    #     if i % 1000 == 0:
    #         print(i)
    #
    # y[:n//2] = np.array([1.0, 0.0])
    # y[n//2:] = np.array([0.0, 1.0])
    #
    # # 
    #optiz = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.1)
    optiz = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    
    #
    
    model.compile(optimizer=optiz,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # # X_train, y_train, X_val, y_val = train_test_split(X, y, test_size=0.1)
    # model.fit(X, y, batch_size=32, epochs=15, validation_split=0.1)

    data_gen = ImageDataGenerator()
    
    train_data_gen = data_gen.flow_from_directory(train_path, (img_width, img_height), shuffle='False', batch_size=32)
    val_data_gen = data_gen.flow_from_directory(test_path, (img_width, img_height), shuffle='False', batch_size=32)
    
    model.summary()
    model = load_model('fine_tuning_dogcat_resnet50')


In [ ]:

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


model.fit_generator(train_data_gen, steps_per_epoch=nb_train_samples, epochs=epochs, validation_data=val_data_gen, validation_steps=nb_validation_samples,  callbacks=callbacks_list)

#model.save('fine_tuning_dogcat_resnet502')

In [ ]:
# Import the required modules
%pylab inline 
import cv2
from .display import clear_output


# Grab the input device, in this case the webcam
# You can also give path to the video file
vid = cv2.VideoCapture("../dataset/source/smorking.mp4")

# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    while(True):
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(frame)
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    #print("Released Video Resource")

In [ ]:
###  https://zulko.github.io/moviepy/getting_started/efficient_moviepy.html

from moviepy.editor import * # imports everything, quick and dirty
import moviepy.editor as mpy # Clean. Then use mpy.VideoClip, etc.
from moviepy.editor import VideoFileClip # just import what you need
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.video.fx.resize import resize

clip = VideoFileClip("../dataset/source/smorking.mp4").rotate(0)
print( clip.duration )
clip.ipython_display(maxduration=666)


In [ ]:
! pip install moviepy

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import ResNet50
from keras.preprocessing.image import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint


import numpy as np
import os

np.random.seed(2017)

from skimage.io import imread_collection, imread
from skimage.transform import resize

filepath='../every.resn50.hdf5'
img_width, img_height = 150, 150

train_data_dir = '../dataset/training_set/smo'
validation_data_dir = '../dataset/test_set/smo'

nb_train_samples = 4521
nb_validation_samples = 770
epochs = 1
batch_size = 16



if __name__ == '__main__':

    base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg',
                          input_tensor=Input((224, 224, 3)))
    
    for layer in base_model.layers:
        layer.trainable = False
        
    # for tmp in zip([x.name for x in base_model.layers], range(len(base_model.layers))):
    #     print(tmp)

    x = Flatten()(base_model.output)
    x = Dense(2, activation='softmax', name='fc2')(base_model.output)

    model = Model(base_model.input, x)

    # for tmp in zip([x.name for x in model.layers], range(len(model.layers))):
    #     print(tmp)

    for layer in model.layers[140:]:
        layer.trainable = True

    # load data
    # now_path = os.path.abspath(os.path.curdir)
    # project_path, tmp = os.path.split(now_path)
    # train_path = os.path.join(project_path, 'train')
    #train_path = '/home/zhangzhe/pycharm/dogcat/train'
    train3_path ='../dataset/training_set/smo'
    val3_path = '../dataset/test_set/smo'

    #n = 25000
    # X = np.zeros((n, 224, 224, 3), dtype=np.uint8)
    # y = np.zeros((n, 2), dtype=np.float32)
    #
    # for i in range(n//2):
    #     X[i] = resize(imread(os.path.join(train_path, 'cat.%d.jpg' % i)), (224, 224, 3))
    #     X[i+n//2] = resize(imread(os.path.join(train_path, 'dog.%d.jpg' % i)), (224, 224, 3))
    #     if i % 1000 == 0:
    #         print(i)
    #
    # y[:n//2] = np.array([1.0, 0.0])
    # y[n//2:] = np.array([0.0, 1.0])
    #
    # # 
    #optiz = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.1)
    optiz = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    #
    model.compile(optimizer=optiz,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # # X_train, y_train, X_val, y_val = train_test_split(X, y, test_size=0.1)
    # model.fit(X, y, batch_size=32, epochs=15, validation_split=0.1)

    data_gen = ImageDataGenerator()
    train_data_gen = data_gen.flow_from_directory(train3_path, (224, 224), shuffle='False', batch_size=32)
    val_data_gen = data_gen.flow_from_directory(val3_path, (224, 224), shuffle='False', batch_size=32)


In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import ResNet50
from keras.preprocessing.image import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint


import numpy as np
import os

np.random.seed(2017)

from skimage.io import imread_collection, imread
from skimage.transform import resize

filepath='../every.resn50.hdf5'
img_width, img_height = 243, 243

train_data_dir = '../dataset/training_set/smo'
validation_data_dir = '../dataset/test_set/smo'

nb_train_samples = 4521
nb_validation_samples = 770
epochs = 3
batch_size = 16



if __name__ == '__main__':

    
    
    base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg'
                       ,input_tensor=Input((img_width, img_height, 3)))
        
    # - https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py
    
    for layer in base_model.layers:
        layer.trainable = False

    #x = Flatten()(base_model.output)
    x = Dense(2, activation='softmax', name='fc2')(base_model.output) #加入神經元(隱藏層) 2,foftmax
 
    model = Model(base_model.input, x)

    # for tmp in zip([x.name for x in model.layers], range(len(model.layers))):
    #     print(tmp)

    for layer in model.layers[140:]:
        layer.trainable = True

    # load data
    # now_path = os.path.abspath(os.path.curdir)
    # project_path, tmp = os.path.split(now_path)
    # train_path = os.path.join(project_path, 'train')
    #train_path = '/home/zhangzhe/pycharm/dogcat/train'
    train_path ='../dataset/training_set/smo'
    test_path = '../dataset/test_set/smo'

    #n = 25000
    # X = np.zeros((n, 224, 224, 3), dtype=np.uint8)
    # y = np.zeros((n, 2), dtype=np.float32)
    #
    # for i in range(n//2):
    #     X[i] = resize(imread(os.path.join(train_path, 'cat.%d.jpg' % i)), (224, 224, 3))
    #     X[i+n//2] = resize(imread(os.path.join(train_path, 'dog.%d.jpg' % i)), (224, 224, 3))
    #     if i % 1000 == 0:
    #         print(i)
    #
    # y[:n//2] = np.array([1.0, 0.0])
    # y[n//2:] = np.array([0.0, 1.0])
    #
    # # 
    #optiz = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.1)
    optiz = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    
    #
    
    model.compile(optimizer=optiz,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    # # X_train, y_train, X_val, y_val = train_test_split(X, y, test_size=0.1)
    # model.fit(X, y, batch_size=32, epochs=15, validation_split=0.1)

    data_gen = ImageDataGenerator()
    
    train_data_gen = data_gen.flow_from_directory(train_path, (img_width, img_height), shuffle='False', batch_size=32)
    val_data_gen = data_gen.flow_from_directory(test_path, (img_width, img_height), shuffle='False', batch_size=32)
    
    model.summary()
    model = load_model('../every.resn50.90130.hdf5')


In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import numpy as np
import pandas as pd
import os
import os
import cv2
import time
  
import numpy as np
from keras.preprocessing import image
from matplotlib import pyplot as plt
import glob
import numpy as np
from keras.preprocessing import image

from io import BytesIO
import PIL
from IPython.display import display, Image

if __name__ == '__main__':
    test_path = '../dataset/source/out/'
    # [cat, dog]
    # yiyi is 1 if the image is a dog, 0 if cat

    #model = load_model('/home/zhangzhe/pycharm/dogcat/src/fine_tuning_dogcat_resnet50')
    #print('model load finished')

    files = os.listdir(test_path)

    prediction = []
    file_indexs = []
    im_ind = 0
    print('Start Prediction: ')
    for file in files:
        print(im_ind, file)
        img_path = os.path.join(test_path, file)
        img_pil = load_img(img_path, target_size=(img_width, img_height))
        img = img_to_array(img_pil)
        img = np.expand_dims(img, axis=0)
        result = model.predict(img)
        file_name, suffix = file.split('.')
        prediction.append(result[0][1])
        file_indexs.append(file_name)
        ans = result[0][1];
        print('show ans= ', ans)
       
        if ans > 0.45:
            
            print ("smorking----------smorking------smorking------------------")
            
            img = cv2.imread(img_path)
            #cv2.imshow(winname='show the image',mat=img) 
            im = PIL.Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))  #OpenCV转换成PIL.Image格式
            #plt.imshow(im)
            bio = BytesIO()
            im.save(bio, format='png')
            display(Image(bio.getvalue(), format='jpg'))
            print ("-------------------------------------------------")
            
        else:
            '''
            print ("no no no smorking------------------------")
            img = cv2.imread(img_path)
            #cv2.imshow(winname='show the image',mat=img) 
            im = PIL.Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))  #OpenCV转换成PIL.Image格式
            #plt.imshow(im)
            bio = BytesIO()
            im.save(bio, format='png')
            display(Image(bio.getvalue(), format='jpg'))
            print ("-------------------------------------------------")
            '''
        print('     ', result[0][1], file_name)
        im_ind += 1
        
    print('Finish prediction. ')

    #df = pd.DataFrame({'id':file_indexs, 'label':prediction})
    #df.to_csv('pred2.csv', index=None)
    #df.head(10)

### load image from video


In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import numpy as np
import pandas as pd
import os
import os
import cv2
import time
  
import numpy as np
from keras.preprocessing import image
from matplotlib import pyplot as plt
import glob
import numpy as np
from keras.preprocessing import image

from io import BytesIO
import PIL
from IPython.display import display, Image

################################################################################    
# Import the required modules
%pylab inline 
import cv2
from IPython.display import clear_output

# Grab the input device, in this case the webcam
# You can also give path to the video file
vid = cv2.VideoCapture("../dataset/source/smorking.mp4")

# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
#plt.figure(figsize = (10,10))
#fig = plt.figure()
#print ("fig = ") 
#<Figure size 432x288 with 0 Axes>
label_no_smorking = "predict: no Smorking"
label_smorking = "predict: Smorking!"
count = 0

#'''
try:
    while(True):
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        #get image here
        
        #save image here
        #img_jpg = np.asarray(frame)
        #cv2.imwrite("../dataset/source/out/cyril_frame%d.jpg" % count, img_jpg)
        #count = count + 1
    
        #predict image here !

        resize = cv2.resize(frame, (img_width, img_height)) 
        resize = np.asarray(resize)
        test_image = image.img_to_array(resize)
        inputarray = test_image[np.newaxis,...] # dimension added to fit input size
        result =  model.predict(inputarray)
        #print(result)
        #print("Predicted=%s", result)
        ans = result[0][1]
        
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        if ans > 0.8: 
        
            # put text into image 
            # https://blog.csdn.net/GAN_player/article/details/78155283
            cv2.putText(frame, label_smorking, (30, 60),  cv2.FONT_HERSHEY_SIMPLEX,0.9, (255,0, 0), 2)
            #cv2.line(影像, 開始座標, 結束座標, 顏色, 線條寬度)
            cv2.line(frame, (0, 0), (432, 0),     (255,0,0),  12)
            cv2.line(frame, (0, 0), (0, 350),     (255,0,0),  12)
            cv2.line(frame, (440, 0),(440, 440),  (255,0,0),  12)
            cv2.line(frame, (0, 330),(440, 330),(255,0,0),12)

        else:
             # put text into image 
            # https://blog.csdn.net/GAN_player/article/details/78155283
            cv2.putText(frame, label_no_smorking, (30, 60),  cv2.FONT_HERSHEY_SIMPLEX,0.9, (255,255, 255), 2)
            #cv2.line(影像, 開始座標, 結束座標, 顏色, 線條寬度)
        
        # Turn off the axis
        plt.axis('off')
        
        # Title of the window
        plt.title("Input Stream")
        # Display the frame
        plt.imshow(frame)
        plt.show()
        # Display the frame until new frame is available
        clear_output(wait=True)
                
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    #print("Released Video Resource")
    

In [ ]:
print(result)

In [ ]:
ans = result[0][1]

In [ ]:
print(ans)